In [ ]:
import bibliograph as bg

bibtex_fname = "bibliograph/test_data/bibtex_test_data_short.bib"
entry_syntax_fname = "bibliograph/resources/default_bibtex_syntax.csv"

tn = bg.slurp_bibtex(
    bibtex_fname,
    entry_syntax_fname,
    syntax_case_sensitive=False,
    space_char='|',
    na_string_values='!',
    na_node_type='missing'
)

tn.strings

In [ ]:
import bibliograph as bg

shorthand_fname='bibliograph/test_data/manual_annotation.shnd'
entry_syntax_fname = "bibliograph/resources/default_entry_syntax.csv"
link_syntax_fname="bibliograph/resources/default_link_syntax.csv"

tn = bg.slurp_shorthand(
    shorthand_fname,
    entry_syntax_fname,
    link_syntax_fname,
    syntax_case_sensitive=False,
    item_separator='__',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    default_entry_prefix='wrk',
    skiprows=2,
    comment_char='#'
)

In [8]:
import bibliograph as bg

shorthand_fname='bibliograph/test_data/shorthand_with_aliases.shnd'
entry_syntax_fname = "bibliograph/resources/default_entry_syntax.csv"
link_syntax_fname="bibliograph/resources/default_link_syntax.csv"

aliases_dict = {
    'actor': 'bibliograph/test_data/aliases_actor.csv',
    'work': 'bibliograph/test_data/aliases_work.csv'
}

tn = bg.slurp_shorthand(
    shorthand_fname,
    entry_syntax_fname,
    link_syntax_fname,
    syntax_case_sensitive=False,
    aliases_dict=aliases_dict,
    item_separator='__',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    default_entry_prefix='wrk',
    skiprows=2,
    comment_char='#',
)

tn.resolve_nodes().query('name_string != abbr_string')

,node_type,name_string,abbr_string,date_inserted,date_modified
0,actor,Alice Smith,asmith,2022-07-22 10:22:56,<NA>
1,actor,Elizabeth Wu,bwu,2022-07-22 10:22:56,<NA>
2,actor,National Aeronautics and Space Administration,nasa,2022-07-22 10:22:56,<NA>
3,actor,some other person,soperson,2022-07-22 10:22:56,<NA>
4,work,j. atmos. sci.,jats,2022-07-22 10:22:56,<NA>


# TODO: alias insertion executes without error, now write test cases

In [ ]:
!pytest bibliograph/tests.py